# 3. 基于MindSpore构造非对称损失函数

 本实验主要介绍非对称损失函数的原理和构造，使用MindSpore构造非对称损失函数，以Focal Loss损失函数作为讲解实例。

## 1、实验目的

- 掌握非对称损失函数的原理。
- 掌握如何使用MindSpore构造非对称损失函数。

## 2、非对称损失函数原理介绍

非对称损失函数可以看做是一种最优化方法，通过控制参数变化，使模型获得适合的拟合行为，从而降低误差和达到最优效果。它是在构建模型权重时，从损失函数中提取出来的一种损失函数，它可以调整模型拟合程度，改善模型预测精确度。

Focal Loss函数：Focal Loss是基于二分类交叉熵损失的。它是一个动态缩放的交叉熵损失，通过一个动态缩放因子，可以动态降低训练过程中易区分样本的权重，从而将重心快速聚焦在那些难区分的样本（有可能是正样本，也有可能是负样本，但都是对训练网络有帮助的样本），公式如下：
$$FL(p_t)=-{{\alpha}_t}(1-p_t)^{\gamma}log(p_t)$$
即通过${{\alpha}_t}$可以抑制正负样本的数量失衡，通过${\gamma}$可以控制简单/难区分样本数量失衡。

${\gamma}$为一个参数，范围在[0,5]， 当${\gamma}$为0时，就变成了二分类交叉熵损失函数。$(1-p_t)^{\gamma}$可以降低易分样本的损失贡献，从而增加难分样本的损失比例。当$p_t$趋向于1，即说明该样本是易区分样本，此时调制因子$(1-p_t)^{\gamma}$是趋向于0，说明对损失的贡献较小，即降低了易区分样本的损失比例；当$p_t$很小，也就是假如某个样本被分到正样本，但是该样本为前景的概率特别小，即被错分到正样本了，此时调制因子$(1-p_t)^{\gamma}$是趋向于1，对loss也没有太大的影响。

Focal Loss的特点：

（1）调制因子$(1-p_t)^{\gamma}$是用来减低易分样本的损失贡献 ，无论是前景类还是背景类，$p_t$越大，就说明该样本越容易被区分，调制因子也就越小。

（2）${{\alpha}_t}$用于调节正负样本损失之间的比例，前景类别使用${{\alpha}_t}$时，对应的背景类别使用$1-{{\alpha}_t}$。

（3）${\gamma}$和${{\alpha}_t}$都有相应的取值范围，他们的取值相互间也是有影响的，在实际使用过程中应组合使用。

## 3、实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.4；Python环境=3.11。


|  硬件平台 |
  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU |Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.4 Python3.11 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 数据准备

在本次实验中，我们使用numpy从均匀分布中随机生成测试数据，从正态分布中随机生成噪声加入因变量y。

### 4.2 数据加载

使用GeneratorDataset，通过迭代列表构造数据集，指定生成数据集的列名为data和lable，使用batch函数指定每个批处理数据包含的数据条目。

In [1]:
# 导入必要的库，包括NumPy和MindSpore
import numpy as np
from mindspore import dataset as ds
import mindspore.nn as nn
import mindspore as ms

# 生成带有标签的数据集
def get_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0) # 生成服从(-10.0, 10.0)范围内的均匀分布的元素，返回值的元素类型为浮点型。       
        #根据公式生成标签 
        y = 1 if x * w + b >0 else 0     
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.int32) 

def create_dataset(num_data, batch_size=16):
    # 加载数据集
    dataset = ds.GeneratorDataset(list(get_data(num_data)), column_names=['data', 'label']) 
    # 每个batch有16个数据
    dataset = dataset.batch(batch_size) 
    return dataset

# 可视化生成的数据
eval_data=list(get_data(5))

x,y=zip(*eval_data) #zip()函数迭代eval_data，将eval_data中的元素打包成一个个元组，然后返回由这些元组组成的列表。

eval_data

[(array([9.9895935], dtype=float32), array([1])),
 (array([-7.1175165], dtype=float32), array([0])),
 (array([7.8569226], dtype=float32), array([1])),
 (array([-7.842265], dtype=float32), array([0])),
 (array([2.4499097], dtype=float32), array([1]))]

## 5、模型构建

### 5.1 导入Python库和模块

在使用前，导入需要的Python库和模块。

In [2]:
# 时间处理模块
import time
# 科学计算库
import numpy as np
# MindSpore库
import mindspore as ms
# 常见算子操作
import mindspore.ops as ops
# 数据集处理模块
from mindspore import dataset as ds
# 神经网络模块，张量，模型编译
from mindspore import Tensor
# 模型训练设置
from mindspore.train import Callback, LossMonitor
# MindSpore环境设置的0号种子
ms.common.set_seed(0)
import mindspore.nn as nn
from mindspore.common.initializer import Normal

### 5.2 定义模型

定义线性回归网络和构造函数。

In [3]:
# 定义线性回归网络
class LinearNet(nn.Cell):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))        
        #将结果变为概率
        self.Sigmoid = nn.Sigmoid()        

    def construct(self, x):        
        return self.Sigmoid(self.fc(x)) 

### 5.3 自定义Focal Loss损失函数

首先定义损失函数，再自定义Focal Loss损失函数。

In [4]:
#定义损失函数
class FocalLossForLabel(nn.LossBase):
    def __init__(self,alpha=0.25, gamma=2.0, reduction="mean"):
        super(FocalLossForLabel, self).__init__(reduction)
        self.abs = ops.abs
        self.alpha = alpha
        self.gamma = gamma

    def construct(self, base, target):
        at = self.alpha*target + (1-self.alpha)*(1-target)
        pt = base*target + (1-base)*(1-target)
        loss = -at*(1-pt)**self.gamma * ops.log(pt)
        return loss

# 自定义Focal Loss损失函数
class FocalLoss(nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(FocalLoss, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn
        
    def construct(self, data, label):
        output = self._backbone(data)
        return self._loss_fn(output, label)

## 6、模型训练

完成数据预处理、网络定义、损失函数之后，选择优化器，开始模型训练。模型训练包含10层迭代，数据集按分组从训练集中抽取数据，输入网络计算得到损失函数。

In [5]:
ds_train = create_dataset(num_data=10000)
net = LinearNet()
# 定义损失函数
loss_fnn = FocalLossForLabel() 
# 定义损失网络，连接前向网络和损失函数
# loss_net = FocalLoss(net, loss)
# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

# 1. Define forward function
def forward_fn(data, label):
    logits = net(data)
    loss = loss_fnn(logits, label)
    return loss, logits

# 2. Get gradient function
grad_fn = ms.value_and_grad(forward_fn, None, opt.parameters, has_aux=True)

# 3. Define function of one-step training
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    opt(grads)
    return loss

def train(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        # print(label)
        loss = train_step(data, label)
        # print(loss.shape)

        if batch % 10 == 0:
            loss, current = loss.asnumpy(), batch
            # print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
epochs = 10
for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(net, ds_train)
print("Done!")

Done!


In [6]:
#输出参数值
for i in net.trainable_params():
    print(np.array(i))

[[Tensor(shape=[], dtype=Float32, value= 2.61318)]]
[Tensor(shape=[], dtype=Float32, value= 3.47061)]


## 7、模型预测

首先随机生成测试数据，再定义标签，然后进行模型测试，最后输出模型的预测结果。

In [7]:
from mindspore import Tensor

# 生成测试数据
w=2.0
b=3.0

x = np.array(-1).reshape(1,1)
x1 = np.array([x]).astype(np.float32)
    
true_result = 1 if x * w + b >0 else 0
print('data:' + '%s'%x)
# 模型测试
test_result = net(Tensor(x1))

print('predict result:' + '%s'%test_result)
print('true result:' + '%s'%true_result)

data:[[-1]]
predict result:[[[0.70212436]]]
true result:1
